## 1. Load NERs extracted previously

In [7]:
import json

with open("../out/prompting_ners_gpt_4.1_mini.json", "r", encoding="utf-8") as fr:
    prompting_ners = json.load(fr)

## 2. Preparing LLM for RE

In [84]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [85]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [86]:
with open("../data/introduccion.txt", "r", encoding="utf-8") as fr:
    text = fr.read()
    
with open("../data/historia_juan_rana.txt", "r", encoding="utf-8") as fr:
    text += fr.read()

### 2.1. First approach: Analyze the entire text passing NERs and text into the prompt

In [88]:
from langchain.prompts import PromptTemplate

entire_text_prompt = PromptTemplate(
    input_variables=["text", "entities"],
    template="""
        Dado el siguiente texto y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre dichas entidades.

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Lista de entidades (NERs):
        {entities}

        Texto de entrada:
        {text}

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
    """,
)

In [89]:
response = llm.invoke(
    entire_text_prompt.invoke({
        "text": text,
        "entities": "\n".join(
            [
                f"{key}: {', '.join(values)}"
                for key, values in prompting_ners.items()
            ]
        ),
    })
).content

In [90]:
relations = set()

for relation in response.split("\n"):
    print(relation.strip())
    relations.add(relation.strip())

(Cosme Pérez)-[fue conocido artísticamente como]-(Juan Rana)
(Cosme Pérez)-[nació en]-(Tudela de Duero)
(Cosme Pérez)-[fue hijo de]-(Damián Pérez)
(Cosme Pérez)-[fue hijo de]-(Isabel de Basto)
(Cosme Pérez)-[estuvo casado con]-(Bernarda Ramírez)
(Cosme Pérez)-[estuvo casado con]-(Bernarda Manuela)
(Cosme Pérez)-[estuvo casado con]-(María de Acosta)
(Cosme Pérez)-[tuvo como hija]-(Francisca María Pérez)
(Cosme Pérez)-[fue recibido en]-(Cofradía de Nuestra Señora de la Novena)
(Cosme Pérez)-[estuvo en compañía de]-(Tomás Fernández [Cabredo])
(Domingo Canejil)-[recibió nombre de]-(Ranilla)
(Cosme Pérez)-[tuvo confianza de]-(Rey Felipe IV)
(Cosme Pérez)-[intervino ante]-(Rey Felipe IV)
(Cosme Pérez)-[tuvo parentesco con]-(Bárbara Coronel)
(Pedro de la Rosa)-[quitó papeles a]-(Juan Rana)
(Juan Rana)-[fue alabado por]-(familia real)
(Cosme Pérez)-[poseyó casas en]-(calle de Cantarranas)
(Juan Caramuel)-[documentó anécdota de]-(Juan Rana)
(Juan Rana)-[representó papel de]-(alcalde rústico)
(C

In [91]:
print(f"Total de relaciones encontradas: {len(response.split("\n"))}, de las cuales {len(relations)} son únicas.")

Total de relaciones encontradas: 151, de las cuales 130 son únicas.


In [92]:
for rel in relations:
    if len(rel.split("-")) >= 3:
        print(rel)

(Juan Navarro Oliver)-[hizo inventario de bienes de]-(Luis López Sustaete)
(Juan Rana)-[actuó en]-(El alcalde de Alcorcón)
(Cosme Pérez)-[actuó ante]-(Su Majestad)
(Cosme Pérez)-[estuvo casado con]-(María de Acosta)
(Juan Rana)-[actuó en]-(La Zarzuela)
(Cosme Pérez)-[actuó con]-(Antonio Ramos)
(Cosme Pérez)-[actuó con]-(María de Quiñones)
(Cosme Pérez)-[actuó con]-(Francisca Bezón)
(Cosme Pérez)-[murió en]-(Madrid)
(Juan Rana)-[actuó en]-(Aguardad, supremos dioses)
(Cosme Pérez)-[formó parte de]-(compañía de Juan Bautista Valenciano)
(Cosme Pérez)-[actuó con]-(Mateo de Godoy)
(Cosme Pérez)-[actuó con]-(Manuela Escamilla)
(Cosme Pérez)-[recibió donativo de]-(Reina Mariana de Austria)
(Juan Rana)-[fue alabado por]-(familia real)
(Cosme Pérez)-[tuvo sobrina]-(María de Diego)
(Felipe IV)-[se refirió a]-(Juan Rana)
(Cosme Pérez)-[tuvo parentesco con]-(Bárbara Coronel)
(Cosme Pérez)-[salió en carro en]-(Fieras afemina amor)
(Bernarda Ramírez)-[actuó con]-(Juan Rana)
(Cosme Pérez)-[actuó en]-

In [93]:
with open("../out/full_text_re.txt", "w", encoding="utf-8") as fw:
    unique_relations = '\n'.join(rel for rel in relations if len(rel.strip().split("-")) >= 3)
    fw.write(unique_relations)

### 2.2. Second approach: Analyze each paragraph

In [94]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [95]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [102]:
import pandas as pd
import ast
    
data = pd.read_csv("../data/DicatJuanRana_w_clarified_sentences.csv", sep=";", encoding="utf-8", converters={"sentences": ast.literal_eval, "clarified_sentences": ast.literal_eval})

In [98]:
from langchain.prompts import PromptTemplate

paragraph_text_prompt = PromptTemplate(
    input_variables=["text", "entities"],
    template="""
        Dado el siguiente texto y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre dichas entidades.

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
        Lista de entidades (NERs):
        {entities}

        Texto de entrada:
        {text}
    """,
)

In [103]:
from tqdm.auto import tqdm
from collections import defaultdict

entities = "\n".join([
    f"{key}: {', '.join(values)}"
    for key, values in prompting_ners.items()
])

total_re_found = 0

per_paragraph = defaultdict(set)

for _, row in tqdm(data.iterrows(), total=len(data)):
    paragraph = "".join(row["clarified_sentences"])

    response = llm.invoke(
        paragraph_text_prompt.invoke({
            "text": paragraph,
            "entities": entities,
        })
    ).content
    
    total_re_found += len(response.split("\n"))

    for relation in response.split("\n"):
        relation = relation.strip()
        if relation and len(relation.split("-")) >= 3:
            per_paragraph[row["year"]].add(relation)

  0%|          | 0/38 [00:00<?, ?it/s]

In [104]:
unique_relations = {relation for relations in per_paragraph.values() for relation in relations}

In [105]:
print(f"Total de relaciones encontradas: {total_re_found}, de las cuales {len(unique_relations)} son únicas.")

Total de relaciones encontradas: 536, de las cuales 519 son únicas.


In [106]:
import json

with open("../out/per_paragraph_text_re.json", "w", encoding="utf-8") as fw:
    json.dump({key: list(values) for key, values in per_paragraph.items()}, fw, ensure_ascii=False, indent=4)

### 2.3. Third approach: Analyze each sentence with overlap

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
import os

In [5]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [6]:
import json

with open("../out/prompting_ners_gpt_4.1_mini.json", "r", encoding="utf-8") as fr:
    prompting_ners = json.load(fr)

In [7]:
import pandas as pd
import ast
    
data = pd.read_csv("../data/DicatJuanRana_w_clarified_sentences.csv", sep=";", encoding="utf-8", converters={"sentences": ast.literal_eval, "clarified_sentences": ast.literal_eval})

In [13]:
from langchain.prompts import PromptTemplate

entire_text_prompt = PromptTemplate(
    input_variables=["text", "prev_context", "entities"],
    template="""
        Dada la siguiente frase y una lista de entidades nombradas previamente extraídas, identifica todas las relaciones explícitas o implícitas entre las entidades que tanto en la frase como en la lista de entidades.
        
        Las frases pueden contener un contexto para ayudar a identificar las relaciones (para frases que comiencen con fue..., el cual..., etc.) pero no se deben sacar relaciones del propio contexto.

        Las relaciones deben expresarse únicamente en el formato:
        (Entidad1)-[relación]-(Entidad2)

        Donde:

        - Entidad1 y Entidad2 deben coincidir con entidades de la lista proporcionada.

        - [relación] debe ser un verbo o una expresión verbal que indique la relación entre las entidades en el contexto del texto.

        - Si una relación puede expresarse con sinónimos más generales o normalizados (por ejemplo, "dirige", "es jefe de" → "dirige"), elige el término más general.

        - Ignora relaciones que no se puedan inferir directamente del texto.

        Salida esperada:
        (EntidadA)-[relación]-(EntidadB)
        (EntidadC)-[relación]-(EntidadD)
        …
        
        Muestra la salida únicamente con las relaciones encontradas, sin ningún otro texto adicional.
        
        Lista de entidades (NERs):
        {entities}

        Contexto de la frase (No se debe usar el contexto para sacar relaciones, pero puede ayudar a entender mejor la frase en situaciones en las que el sujeto o el objeto de la relación no estén claros):
        El texto forma parte de la biografía de Cosme Pérez. 
        
        - Contexto previo a la frase: {prev_context}

        Frase de entrada:
        {text}
    """,
)

In [18]:
from tqdm.auto import tqdm
from collections import defaultdict

entities = "\n".join([
    f"{key}: {', '.join(values)}"
    for key, values in prompting_ners.items()
    if key in ["PER", "LOC", "EVENT", "WORK_OF_ART", "ORG", "GPE"]
])

sentences_of_context = 3 # Number of sentences to consider as context for each sentence

total_re_found = 0
relations = set()

per_sentence = defaultdict(set)
years_to_test = [1636, 1643, 1648, 1651, 1653, 1656, 1665, 1666, 1670]


# for _, year in tqdm(data.iterrows(), total=len(data), position=0):
# for _, year in tqdm(data[(data["year"].astype(int) == 1653) | (data["year"].astype(int) == 1656) | (data["year"].astype(int) == 1670)].iterrows(), position=0):
# for _, year in tqdm(data[data["year"].astype(int) == 1670].iterrows(), position=0):
for _, year in tqdm(data[data["year"].astype(int).isin(years_to_test)].iterrows(), total=len(years_to_test), position=0):
        
    context = []
    sentences = year["sentences"]
    
    for sentence in tqdm(sentences, position=1, leave=True):

        response = llm.invoke(
            entire_text_prompt.invoke({
                "text": sentence.strip(),
                "prev_context": "\n".join(context),
                # "next_context": "\n".join(sentences[i+1:i+1+sentences_of_context]),
                "entities": entities,
            })
        ).content
    
        total_re_found += len(response.split("\n"))

        # for relation in response.split("\n"):
        #     relation = relation.strip()
        #     if relation and len(relation.split("-")) >= 3:
        #         relations.add(relation)
        
        for relation in response.split("\n"):
            relation = relation.strip()
            per_sentence[year["year"]].add(relation)
        
        # DEBUGGING 
        # print("Relaciones encontradas:\n", response.strip())
        # print(f"Frase: {sentence.strip()}")
        # print(f"Contexto previo: {' '.join(context)}")
        # print(f"Contexto posterior: {' '.join(sentences[i+1:i+1+sentences_of_context])}")
        # print()
        
        context.append(sentence.strip())
        if len(context) > sentences_of_context:
            context = context[-sentences_of_context:]  # Keep only the last 'sentences_of_context' sentences

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# from tqdm.auto import tqdm
# from collections import deque

# entities = "\n".join([
#     f"{key}: {', '.join(values)}"
#     for key, values in prompting_ners.items()
# ])

# sentences_of_context = 2 # Number of sentences to consider as context for each sentence
# total_sentences = sum(len(paragraph.split(".")) for paragraph in text.split("\n\n") if paragraph.strip())

# total_re_found = 0
# relations = set()

# for paragraph in tqdm(text.split("\n\n"), position=0):
#     if not paragraph.strip():
#         continue
        
#     context = deque([])
    
#     for sentence in tqdm(paragraph.split("."), position=1, leave=False):
#         if not sentence.strip():
#             continue

#         response = llm.invoke(
#             entire_text_prompt.invoke({
#                 "text": sentence.strip(),
#                 "context": "\n".join(context),
#                 "entities": entities,
#             })
#         ).content
    
#         total_re_found += len(response.split("\n"))

#         for relation in response.split("\n"):
#             relation = relation.strip()
#             if relation and len(relation.split("-")) >= 3:
#                 relations.add(relation)
        
#         context.append(sentence.strip())
#         if len(context) > sentences_of_context:
#             context.popleft()

In [19]:
unique_relations = {relation for relations in per_sentence.values() for relation in relations}

In [20]:
print(f"Total de relaciones encontradas: {total_re_found}, de las cuales {len(unique_relations)} son únicas.")

Total de relaciones encontradas: 182, de las cuales 180 son únicas.


In [5]:
import rich as rc

rc.print(data[data["year"].astype(int) == 1670]["clarified_sentences"].values[0])

[
    'El 9 de julio, en Madrid, firmó un nuevo testamento que invalidaba el anterior de 1666, en el que declaraba de
nuevo ser vecino de Madrid, "que vivo en mis casas propias, en la calle de Cantarranas, Parroquia de San Sebastián,
estando en pie levantado de la cama, aunque con algunos achaques", y pedía ser sepultado "sin ataúd en el convento 
de religiosos descalzos de la Santísima Trinidad desta Corte, en la sepultura que allí tengo mía propia, y donde 
está sepultada mi hija Francisca María Pérez".  \n\nAparte del dinero estipulado para misas por su alma y detalles 
de sus honras, entre las limosnas entregaba una de 150 dcs. de vellón para la Cofradía de Nuestra Señora de la 
Novena para ayuda de la fábrica de su capilla, y un pabellón de brocado que tenía en el oratorio de su casa.  \n\nA
su sobrina María de Diego y a su sobrino Francisco de Diego, ya mencionados en el testamento anterior, les enviaba 
150 dcs. y 30 dcs. de vellón, respectivamente, y 150 dcs. de vellón para ayuda del retablo "que me han dicho se 
trata de hacer" para la imagen de Nuestra Señora del Amparo de la iglesia parroquial de Tudela del Duero.  
\n\n[Nombre de la persona que firmó el testamento] asistió al cabildo de la Cofradía de Nuestra Señora de la Novena
celebrado el 2 de marzo.',
    'El 9 de julio, en Madrid, [Nombre de la persona que firmó el testamento] firmó un nuevo testamento que 
invalidaba el anterior de 1666, en el que [Nombre de la persona que firmó el testamento] declaraba de nuevo ser 
vecino de Madrid, "que vivo en mis casas propias, en la calle de Cantarranas, Parroquia de San Sebastián, estando 
en pie levantado de la cama, aunque con algunos achaques", y pedía ser sepultado "sin ataúd en el convento de 
religiosos descalzos de la Santísima Trinidad desta Corte, en la sepultura que allí tengo mía propia, y donde está 
sepultada mi hija Francisca María Pérez".',
    'Aparte del dinero estipulado para misas por el alma de [Nombre de la persona que firmó el testamento] y 
detalles de sus honras, entre las limosnas entregaba una de 150 dcs. de vellón para la Cofradía de Nuestra Señora 
de la Novena para ayuda de la fábrica de su capilla, y un pabellón de brocado que tenía en el oratorio de su 
casa.',
    'A su sobrina María de Diego y a su sobrino Francisco de Diego, ya mencionados en el testamento anterior, les 
enviaba 150 dcs. y 30 dcs. de vellón, respectivamente, y enviaba 150 dcs. de vellón para ayuda del retablo "que me 
han dicho se trata de hacer" para la imagen de Nuestra Señora del Amparo de la iglesia parroquial de Tudela del 
Duero.',
    'Asimismo, ordenaba pagar a su ama Isabel 150 rs. de vellón y el salario que se le debiera, y a Felipe Negro, 
su "paisano y pariente", otros 150 rs. y un espadín.',
    'Otorgaba a sus albaceas y testamentarios poder para rematar sus bienes y pagar todo lo estipulado en el 
testamento y, con el sobrante, pagar nuevas misas y limosnas para el alma de Cosme Pérez y la de su hija y 
familiares.',
    '—A. de la Granja, al publicar el testamento de Cosme Pérez, estimó que el desembolso en las misas previstas 
por el alma de Cosme Pérez debió de rondar los 7.500 rs. y destaca la preocupación religiosa que manifiesta Cosme 
Pérez—.'
]

In [21]:
unique_relations

{'',
 '(150 dcs. de vellón)-[ayuda]-(retablo)',
 '(A. de la Granja)-[estima desembolso]-(misas previstas por su alma)',
 '(A. de la Granja)-[publica]-(testamento)',
 '(A. de la Granja)-[publicó]-(testamento)',
 '(Agustín Merlo)-[fue detenido por mantener una reyerta con]-(Valdés)',
 '(Andrómeda y Perseo)-[escrita por]-(Calderón)',
 '(Antonio García de Prado)-[otorgó poder a]-(Cosme Pérez)',
 '(Antonio García de Prado)-[otorgó poder a]-(Francisco Ortiz)',
 '(Antonio de Escamilla)-[debía]-(Cosme Pérez)',
 '(Antonio de Solís)-[escribió]-(loa)',
 '(Antonio de Solís)-[escribió]-(sainetes)',
 '(Baccio del Bianco)-[describe]-(fiesta)',
 '(Baccio)-[describe]-(loa)',
 '(Bernarda Ramírez)-[interpretó el papel del "Alma" de]-(Juan Rana)',
 '(Bernarda Ramírez)-[no descuida]-(Juan Rana)',
 '(Bernarda Ramírez)-[participó en]-(Darlo todo y no dar nada)',
 '(Bianco)-[denomina]-(Juan Rana)',
 '(Catalina de Nicolás y la Rosa)-[contrató]-(Cosme Pérez)',
 '(Catalina de Nicolás y la Rosa)-[se comprometía a

In [22]:
import json

with open("../out/per_sentence_text_re.json", "w", encoding="utf-8") as fw:
    json.dump({key: list(values) for key, values in per_sentence.items()}, fw, ensure_ascii=False, indent=4)

# Test de la mejor combinación

Clarified Text + prev + post

```{'(Andrómeda y Perseo)-[es obra de]-(Calderón)',
 '(Andrómeda y Perseo)-[fue escrita por]-(Calderón)',
 '(Andrómeda y Perseo)-[se representó en]-(Coliseo del Buen Retiro)',
 '(Conde de Altamira)-[transmitió respuesta a]-(Cosme Pérez)',
 '(Cosme Pérez)-[alias]-(Juan Rana)',
 '(Cosme Pérez)-[es alias de]-(Juan Rana)',
 '(Cosme Pérez)-[recibió]-(1.000 rs.)',
 '(Cosme Pérez)-[representó el papel de]-(Bato)',
 '(Cosme Pérez)-[solicitó]-(paso de una ración ordinaria)',
 '(Felipe IV)-[daba cuenta a]-(Luisa Enríquez Manrique)',
 '(Francisca María Pérez)-[es hija de]-(Cosme Pérez)',
 '(Francisca María Pérez)-[no puede gozar ración si]-(anda en la farsa)',
 '(Juan Rana)-[cumplió con]-(Andrómeda y Perseo)',
 '(ración ordinaria)-[goza por]-(casa de la Reyna nuestra señora)',
 '(ración ordinaria)-[para]-(Francisca María Pérez)'}
```

Clarified Text + prev

```{'(Andrómeda y Perseo)-[es obra de]-(Calderón de la Barca)',
 '(Andrómeda y Perseo)-[es una comedia de]-(Calderón)',
 '(Andrómeda y Perseo)-[escrita por]-(Calderón)',
 '(Andrómeda y Perseo)-[se representó en]-(Coliseo del Buen Retiro)',
 '(Conde de Altamira)-[transmite respuesta a]-(Cosme Pérez)',
 '(Cosme Pérez)-[alias]-(Juan Rana)',
 '(Cosme Pérez)-[es alias de]-(Juan Rana)',
 '(Cosme Pérez)-[goza]-(casa de la Reyna nuestra señora)',
 '(Cosme Pérez)-[recibió]-(1.000 rs.)',
 '(Cosme Pérez)-[representó el papel de]-(Bato)',
 '(Cosme Pérez)-[solicita]-(Conde de Altamira)',
 '(Cosme Pérez)-[tiene hija]-(Francisca María Pérez)',
 '(Felipe IV)-[daba cuenta a]-(Luisa Enríquez Manrique)',
 '(Francisca María Pérez)-[goza]-(casa de la Reyna nuestra señora)',
 '(Juan Rana)-[ha cumplido con]-(sus obligaciones)'}
```
Clarified Text

```{'(Andrómeda y Perseo)-[es obra de]-(Calderón)',
 '(Andrómeda y Perseo)-[fue escrita por]-(Calderón)',
 '(Andrómeda y Perseo)-[se representó en]-(Coliseo del Buen Retiro)',
 '(Conde de Altamira)-[transmitió respuesta a]-(Cosme Pérez)',
 '(Cosme Pérez)-[es alias de]-(Juan Rana)',
 '(Cosme Pérez)-[recibió]-(fiestas del Corpus de Madrid)',
 '(Cosme Pérez)-[representó el papel de]-(Bato)',
 '(Cosme Pérez)-[tiene hija]-(Francisca María Pérez)',
 '(Felipe IV)-[daba cuenta a]-(Luisa Enríquez Manrique)',
 '(Juan Rana)-[ha cumplido con]-Andrómeda y Perseo',
 '(casa de la Reyna nuestra señora)-[proporciona ración a]-(Francisca María Pérez)'}
```
Texto Normal

```{'(Andrómeda y Perseo)-[es obra de]-(Calderón)',
 '(Comedia)-[se representó en]-(Coliseo del Buen Retiro)',
 '(Conde de Altamira)-[transmitió respuesta a]-(Cosme Pérez)',
 '(Cosme Pérez)-[alias]-(Juan Rana)',
 '(Cosme Pérez)-[recibió]-(fiestas del Corpus de Madrid)',
 '(Cosme Pérez)-[representó el papel de]-(Bato)',
 '(Cosme Pérez)-[solicitó]-(Conde de Altamira)',
 '(Cosme Pérez)-[tiene hija]-(Francisca María Pérez)',
 '(Felipe IV)-[daba cuenta a]-(Luisa Enríquez Manrique)',
 '(Francisca María Pérez)-[recibe ración ordinaria de]-(casa de la Reyna nuestra señora)',
 '(Juan Rana)-[ha cumplido]-(obligaciones)'}
```
Texto Normal + prev

```{'(Andrómeda y Perseo)-[fue obra de]-(Calderón)',
 '(Comedia)-[se representó en]-(Coliseo del Buen Retiro)',
 '(Conde de Altamira)-[transmitió respuesta a]-(Cosme Pérez)',
 '(Cosme Pérez)-[alias]-(Juan Rana)',
 '(Cosme Pérez)-[recibió]-(1.000 rs.)',
 '(Cosme Pérez)-[representó el papel de]-(Bato)',
 '(Cosme Pérez)-[representó]-(Bato)',
 '(Cosme Pérez)-[solicitó]-(paso de una ración ordinaria)',
 '(Felipe IV)-[daba cuenta a]-(Luisa Enríquez Manrique)',
 '(Francisca María Pérez)-[es hija de]-(Cosme Pérez)',
 '(Juan Rana)-[cumplió]-(obligaciones)',
 '(Juan Rana)-[ha cumplido con]-(obligaciones)',
 '(paso de una ración ordinaria)-[goza por]-(casa de la Reyna nuestra señora)',
 '(paso de una ración ordinaria)-[para]-(Francisca María Pérez)'}
```
Texto Normal + prev + post

```{'(Andrómeda y Perseo)-[fue escrita por]-(Calderón)',
 '(Comedia)-[fue representada en]-(Coliseo del Buen Retiro)',
 '(Conde de Altamira)-[transmitió respuesta a]-(Cosme Pérez)',
 '(Cosme Pérez)-[alias]-(Juan Rana)',
 '(Cosme Pérez)-[es]-(Juan Rana)',
 '(Cosme Pérez)-[recibió]-(1.000 rs.)',
 '(Cosme Pérez)-[representó el papel de]-(Bato)',
 '(Cosme Pérez)-[representó]-(Bato)',
 '(Cosme Pérez)-[solicitó]-(Conde de Altamira)',
 '(Cosme Pérez)-[solicitó]-(paso de una ración ordinaria)',
 '(Cosme Pérez)-[tiene hija]-(Francisca María Pérez)',
 '(Felipe IV)-[daba cuenta a]-(Luisa Enríquez Manrique)',
 '(Francisca María Pérez)-[recibe ración ordinaria de]-(casa de la Reyna nuestra señora)',
 '(Juan Rana)-[cumplió]-(obligaciones)',
 '(pasó de una ración ordinaria)-[goza por]-(casa de la Reyna nuestra señora)',
 '(pasó de una ración ordinaria)-[para]-(Francisca María Pérez)'}
```